**Using `secret = xd`**

In [1]:
from dotenv import load_dotenv
import os

load_dotenv()
secret = os.getenv('SECRET_KEY')
print(f"secret:{secret}")

secret:this can be anything. update it later.


In [2]:
import requests
import time 
import hashlib

In [3]:

def good_hash_header():
    tt = str(time.time_ns())
    hash_ = hashlib.new('sha512')
    hash_.update(f"{tt}-{secret}".encode())
    header = {'time-token': tt, 'auth-key': hash_.hexdigest()}
    return header
    

### all clear test case

In [4]:
url = 'http://127.0.0.1:8000/find_jobs'
body = {
  "lat": 0,
  "lng": 0
}
print(body)

#use the 'headers' parameter to set the HTTP headers:
x = requests.post(url, json = body, headers = good_hash_header())

print(x.text)

{'lat': 0, 'lng': 0}
{"job_ids":[10,11,1,11]}


### wrong/no header test case

In [23]:
url = 'http://127.0.0.1:8000/find_jobs'
body = {
  "lat": -32,
  "lng": 4
}
print(body)

#header = good_hash_header() 
#time.sleep(6)
x = requests.post(url, json = body)

print(x.text)

{'lat': -32, 'lng': 4}
{"detail":"Unauthorized 1"}


### late hash test case

In [24]:
url = 'http://127.0.0.1:8000/find_jobs'
body = {
  "lat": -32,
  "lng": 4
}
print(body)

header = good_hash_header() 
time.sleep(6)
x = requests.post(url, json = body, headers = header)

print(x.text)

{'lat': -32, 'lng': 4}
{"detail":"Unauthorized 2"}


### duplicate hash test case

In [25]:
url = 'http://127.0.0.1:8000/find_jobs'
body = {
  "lat": -32,
  "lng": 4
}
print(body)

header = good_hash_header()
x = requests.post(url, json = body, headers = header)
x = requests.post(url, json = body, headers = header)

print(x.text)

{'lat': -32, 'lng': 4}
{"detail":"Unauthorized 3"}


### bad hash test case

In [26]:
url = 'http://127.0.0.1:8000/find_jobs'
body = {
  "lat": -32,
  "lng": 4
}
print(body)

header = good_hash_header()
header['time-token'] = str(time.time_ns()) # by the time this runs, it mismatches the hash
x = requests.post(url, json = body, headers = header)

print(x.text)

{'lat': -32, 'lng': 4}
{"detail":"Unauthorized 4"}


In [27]:
url = 'http://127.0.0.1:8000/find_jobs'
body = {
  "lat": -32,
  "lng": 4
}
print(body)
secret, secret_temp = 'NOT LONGER XD', secret
header = good_hash_header()
header['time-token'] = str(time.time_ns()) # by this runs, it will mismatches the hash
x = requests.post(url, json = body, headers = header)

secret, secret_temp = secret_temp, secret
print(x.text)

{'lat': -32, 'lng': 4}
{"detail":"Unauthorized 4"}


### edit functionality testing 

In [5]:
url = 'http://127.0.0.1:8000/add_job'
body = {'id_': 11,
  "lat": -32,
  "lng": 4
}
print(body)

#use the 'headers' parameter to set the HTTP headers:
x = requests.post(url, json = body, headers = good_hash_header())

print(x.text)

url = 'http://127.0.0.1:8000/edit_job'
body = { 'id_': 11,
"lat_old": -32,
"lng_old": 4,
"lat_new": 0,
"lng_new": 0
}
print(body)

#use the 'headers' parameter to set the HTTP headers:
x = requests.post(url, json = body, headers = good_hash_header())

print(x.text)

{'id_': 11, 'lat': -32, 'lng': 4}
{"message":3}
{'id_': 11, 'lat_old': -32, 'lng_old': 4, 'lat_new': 0, 'lng_new': 0}
{"message":"11 no longer in 87ad4536cffffff. 11 added to 87754e64dffffff. Length of 87754e64dffffff now 14"}


In [32]:
url = 'http://127.0.0.1:8000/add_job'
body = {'id_': 11,
  "lat": -32,
  "lng": 4
}
print(body)

#use the 'headers' parameter to set the HTTP headers:
x = requests.post(url, json = body, headers = good_hash_header())

print(x.text)

url = 'http://127.0.0.1:8000/del_job'
body = { 'id_': 11,
"lat": -32,
"lng": 4,
}
print(body)

#use the 'headers' parameter to set the HTTP headers:
x = requests.post(url, json = body, headers = good_hash_header())

print(x.text)

{'id_': 11, 'lat': -32, 'lng': 4}
{"message":3}
{'id_': 11, 'lat': -32, 'lng': 4}
{"message":"11 no longer in 87ad4536cffffff."}


## analytics testing

In [13]:
url = 'http://127.0.0.1:8000/submit_analytics'
body = {'feeds':  ## rename this?
            {
            0:{'impressions':[0,1,2], 'applies':[0], 'view':[0], 'save':[0]},
            1:{'impressions':[3], 'applies':[], 'view':[], 'save':[3]},
            }
       }
print(body)

#use the 'headers' parameter to set the HTTP headers:
x = requests.post(url, json = body, headers = good_hash_header())

print(x.text)

{'feeds': {0: {'impressions': [0, 1, 2], 'applies': [0], 'view': [0], 'save': [0]}, 1: {'impressions': [3], 'applies': [], 'view': [], 'save': [3]}}}
{"mesage":"Received"}


In [14]:
import sqlalchemy

In [22]:
eng = sqlalchemy.create_engine('sqlite:///fake_main_behaviors_new.db')

In [27]:
from sqlalchemy.sql import text


In [32]:

t = text('SELECT user_id, gig_advertisement_id FROM gig_hires WHERE created_at > :latest').bindparams(latest=2)
#eng.execute(t).fetchall()

[(7, 15),
 (61, 92),
 (60, 21),
 (73, 90),
 (73, 18),
 (8, 83),
 (2, 13),
 (61, 81),
 (74, 36)]